In [1]:
from tensorflow.keras import backend as K
K.clear_session()

In [2]:
!pip install rarfile

In [3]:
import gdown
import rarfile
import os

# Correct URL after permissions are updated
url = 'https://drive.google.com/uc?id=1FAUSzwEDHfLN_McqFpQ6a3VWoc6h9qgw'
output_rar = '/content/WHEAT.rar'

# Download the RAR file
gdown.download(url, output_rar, quiet=False)

# Check if the downloaded file is valid before extracting
if os.path.exists(output_rar):
    try:
        # Try to open the downloaded file as a RAR file
        with rarfile.RarFile(output_rar, 'r') as rar_ref:
            rar_ref.extractall('/content')
        print("File extracted successfully!")
    except rarfile.NotRarFile:
        print("Error: The downloaded file is not a valid RAR archive.")
else:
    print("Error: File not found.")

Downloading...
From (original): https://drive.google.com/uc?id=1FAUSzwEDHfLN_McqFpQ6a3VWoc6h9qgw
From (redirected): https://drive.google.com/uc?id=1FAUSzwEDHfLN_McqFpQ6a3VWoc6h9qgw&confirm=t&uuid=c7c59158-c8e9-4d42-a0bc-059a11611be5
To: /content/WHEAT.rar
100%|██████████| 228M/228M [00:04<00:00, 46.1MB/s]


File extracted successfully!


In [4]:
# Path to the extracted dataset folder

dataset = '/content/Wheat'

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Conv2D, MaxPooling2D, DepthwiseConv2D, BatchNormalization, LeakyReLU, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd

# Directory for dataset
data_dir = dataset  # Replace with your actual dataset path

# Function to create InceptionV3 architecture with Transfer Learning
def inceptionv3_base(input_shape=(224, 224, 3)):
    # Load Pretrained InceptionV3 Model
    base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze base model layers
    for layer in base_model.layers[:249]:
        layer.trainable = False
    for layer in base_model.layers[249:]:
        layer.trainable = True

    # Extract features
    x = base_model.output
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = GlobalAveragePooling2D()(x)

    return Model(base_model.input, x)

# Function to create Custom-CNN from scratch
def custom_cnn_base(input_shape=(224, 224, 3)):
    inputs = layers.Input(shape=input_shape)

    # Block 1
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Depthwise Separable Convolution Blocks
    x = DepthwiseConv2D((3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(64, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Block 3
    x = DepthwiseConv2D((3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(128, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Global Average Pooling at the end
    x = GlobalAveragePooling2D()(x)
    return Model(inputs, x)

# Build the feature extraction parts
inception_base = inceptionv3_base(input_shape=(224, 224, 3))
custom_cnn_base = custom_cnn_base(input_shape=(224, 224, 3))

# Define input
inputs = layers.Input(shape=(224, 224, 3))

# Extract features from both models
inception_features = inception_base(inputs)
custom_cnn_features = custom_cnn_base(inputs)

# Concatenate both feature vectors
x = Concatenate()([inception_features, custom_cnn_features])

# Fully connected layers
x = Dense(1024, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)

# Create hybrid model with transfer learning
hybrid_model = Model(inputs, outputs)

# Compile the hybrid model
hybrid_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Image Data Generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(data_dir, 'validation'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)

# Train the hybrid model
history = hybrid_model.fit(train_generator, epochs=50, validation_data=val_generator)

# Evaluate model on test data
def evaluate_model(test_generator, model, classes, title="Performance"):
    y_test_pred = model.predict(test_generator)
    y_test_pred = np.argmax(y_test_pred, axis=1)

    precision = precision_score(test_generator.labels, y_test_pred, average='weighted')
    recall = recall_score(test_generator.labels, y_test_pred, average='weighted')
    f1 = f1_score(test_generator.labels, y_test_pred, average='weighted')
    accuracy = accuracy_score(test_generator.labels, y_test_pred)

    print(f"\n{title}")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    # Display confusion matrix
    confusion = confusion_matrix(test_generator.labels, y_test_pred)
    confusion_df = pd.DataFrame(confusion, index=classes, columns=classes)
    print("Confusion Matrix:\n", confusion_df)

# Test the hybrid model on test set
evaluate_model(test_generator, hybrid_model, ['Brown_Rust', 'Healthy', 'Yellow_Rust'], title="Performance on Test Data")


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Found 4204 images belonging to 3 classes.
Found 598 images belonging to 3 classes.
Found 1198 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 85s 355ms/step - accuracy: 0.8051 - loss: 0.4618 - val_accuracy: 0.8963 - val_loss: 0.2540
Epoch 2/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 30s 121ms/step - accuracy: 0.9900 - loss: 0.0336 - val_accuracy: 0.9548 - val_loss: 0.1283
Epoch 3/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 16s 123ms/step - accuracy: 0.9966 - loss: 0.0103 - val_accuracy: 0.9666 - val_loss: 0.1129
Epoch 4/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 21s 123ms/step - accuracy: 0.9969 - loss: 0.0126 - val_accuracy: 0.9666 - val_loss: 0.1239
Epoch 5/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 18s 134ms/step - accuracy: 0.9992 - loss: 0.0042 - val_accuracy: 0.9732 - val_loss: 0.1032
Epoch 6/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 126ms/step - accuracy: 0.9969 - loss: 0.0078 - val_accuracy: 0.9615 - val_loss: 0.1608
Epoch 7/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 126ms/step - accuracy: 0.9983 - loss: 0.0069 - val_accuracy: 0.9615 - val_loss: 0.1394
Epoch 8/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 128ms/step - accuracy: 0.9977 - loss: 0

In [ ]:
# Save the model weights with the correct naming convention
weights_path = "/content/hybrid_model_weights.weights.h5"  # File name ending with .weights.h5
hybrid_model.save_weights(weights_path)
print(f"Weights saved to {weights_path}")

# Download the weights file
from google.colab import files
files.download(weights_path)  # Ensure the correct file path


Weights saved to /content/hybrid_model_weights.weights.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>